In [7]:
from google.auth.transport import requests

# Imports a module to allow authentication using Application Default Credentials (ADC)
import google.auth

In [8]:
def dicomweb_store_instance(project_id, location, dataset_id, dicom_store_id, dcm_file):
    """Handles the POST requests specified in the DICOMweb standard.

    See https://github.com/GoogleCloudPlatform/python-docs-samples/tree/main/healthcare/api-client/v1/dicom
    before running the sample."""

    # Imports the google.auth.transport.requests transport
    # Gets credentials from the environment. google.auth.default() returns credentials and the
    # associated project ID, but in this sample, the project ID is passed in manually.
    credentials, _ = google.auth.default()

    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )
    # Creates a requests Session object with the credentials.
    session = requests.AuthorizedSession(scoped_credentials)

    # URL to the Cloud Healthcare API endpoint and version
    base_url = "https://healthcare.googleapis.com/v1"

    url = f"{base_url}/projects/{project_id}/locations/{location}"

    dicomweb_path = "{}/datasets/{}/dicomStores/{}/dicomWeb/studies".format(
        url, dataset_id, dicom_store_id
    )

    with open(dcm_file, "rb") as dcm:
        dcm_content = dcm.read()

    # Sets required "application/dicom" header on the request
    headers = {"Content-Type": "application/dicom"}

    response = session.post(dicomweb_path, data=dcm_content, headers=headers)
    response.raise_for_status()
    print("Stored DICOM instance:")
    print(response.text)
    return response



In [40]:
from glob import glob
import os

project_id = 'idc-external-031'  # replace with your GCP project ID
location = 'europe-west2'  # replace with the parent dataset's location
dataset_id = 'kaapana-integration-test'  # replace with the parent dataset's ID
dicom_store_id = 'kaapana' # replace with the DICOM store ID

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ubuntu/idc-external-031-f4c357d3d6b2.json"
dcm_files = glob('/home/ubuntu/TestData/nsclc*/**/*.dcm', recursive=True)

for dcm_file in dcm_files:
    dicomweb_store_instance(project_id=project_id, location=location, dataset_id=dataset_id, dicom_store_id=dicom_store_id, dcm_file=dcm_file)

Stored DICOM instance:
<NativeDicomModel><DicomAttribute tag="00081190" vr="UR" keyword="RetrieveURL"><Value number="1">https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana/dicomWeb/studies/1.3.6.1.4.1.32722.99.99.203715003805996641695765332389135385095</Value></DicomAttribute><DicomAttribute tag="00081199" vr="SQ" keyword="ReferencedSOPSequence"><Item number="1"><DicomAttribute tag="00081150" vr="UI" keyword="ReferencedSOPClassUID"><Value number="1">1.2.840.10008.5.1.4.1.1.2</Value></DicomAttribute><DicomAttribute tag="00081155" vr="UI" keyword="ReferencedSOPInstanceUID"><Value number="1">1.3.6.1.4.1.32722.99.99.310628096189403179977370889007298713661</Value></DicomAttribute><DicomAttribute tag="00081190" vr="UR" keyword="RetrieveURL"><Value number="1">https://healthcare.googleapis.com/v1/projects/idc-external-031/locations/europe-west2/datasets/kaapana-integration-test/dicomStores/kaapana/dicomWeb/

In [37]:
import pydicom
from collections import defaultdict

In [38]:
from tqdm.auto import tqdm

series_dict = defaultdict(list)
for dcm_file in tqdm(dcm_files):
    try:
        # Read the DICOM file
        dicom_data = pydicom.dcmread(dcm_file)
        
        # Get SeriesInstanceUID and group files
        series_uid = dicom_data.get("SeriesInstanceUID", "UnknownSeries")
        series_dict[series_uid].append(dicom_data)
    except Exception as e:
        print(f"Error reading {dcm_file}: {e}")

100%|██████████| 1037/1037 [00:05<00:00, 199.87it/s]


In [39]:
# Analyze each series
filtered_series = []
for series_uid, dicom_files in series_dict.items():
    try:
        # Extract metadata from the first file in the series (assuming consistency)
        first_file = dicom_files[0]
        patient_name = first_file.get("PatientName", "N/A")
        patient_sex = first_file.get("PatientSex", "N/A")
        study_date = first_file.get("StudyDate", "N/A")
        series_description = first_file.get("SeriesDescription", "N/A")
        study_description = first_file.get("StudyDescription", "N/A")
        num_instances = len(dicom_files)
        
        # Print series information
        print(f"Series UID: {series_uid}")
        print(f"Patient Name: {patient_name}")
        print(f"Patient Sex: {patient_sex}")
        print(f"Study Date: {study_date}")
        print(f"Series Description: {series_description}")
        print(f"Study Description: {study_description}")
        print(f"Number of Instances: {num_instances}")
        print("-" * 50)
        
        # Filter criteria (example: number of instances > 10 and a specific Series Description)
        if num_instances > 10 and "CT" in series_description:
            filtered_series.append({
                "Series UID": series_uid,
                "Patient Name": patient_name,
                "Patient Sex": patient_sex,
                "Study Date": study_date,
                "Series Description": series_description,
                "Study Description": study_description,
                "Number of Instances": num_instances,
            })
    except Exception as e:
        print(f"Error analyzing series {series_uid}: {e}")

# Print filtered series
print("Filtered Series:")
for series in filtered_series:
    print(series)

Series UID: 1.3.6.1.4.1.32722.99.99.232988001551799080335895423941323261228
Patient Name: LUNG1-002
Patient Sex: M
Study Date: 20140101
Series Description: N/A
Study Description: N/A
Number of Instances: 111
--------------------------------------------------
Series UID: 1.2.276.0.7230010.3.1.3.2323910823.4780.1597260528.760
Patient Name: LUNG1-004
Patient Sex: M
Study Date: 20060924
Series Description: Segmentation
Study Description: N/A
Number of Instances: 1
--------------------------------------------------
Series UID: 1.3.6.1.4.1.32722.99.99.280981614462592634652021339314828620785
Patient Name: LUNG1-004
Patient Sex: M
Study Date: 20060924
Series Description: N/A
Study Description: N/A
Number of Instances: 114
--------------------------------------------------
Series UID: 1.3.6.1.4.1.32722.99.99.234266984120913576066174046169649067872
Patient Name: LUNG1-007
Patient Sex: M
Study Date: 20100528
Series Description: N/A
Study Description: N/A
Number of Instances: 129
-----------------